In [129]:
import sys
sys.path.append("..") 
from utils.surface_gases import *

In [130]:
COUPLER_options = {}

COUPLER_options['Phi_global'] = 1.0
COUPLER_options['T_outgas'] = 2700.0

COUPLER_options['radius'] = 6.371e6 * 1.875
COUPLER_options['mass']   = 5.972e24 * 7.99
COUPLER_options['gravity'] = 6.67e-11 * COUPLER_options['mass'] / (COUPLER_options['radius']**2)
COUPLER_options['hydrogen_earth_oceans'] = 8.0
COUPLER_options['CH_ratio'] = 1.0
COUPLER_options['fO2_shift_IW'] = 0.0
COUPLER_options['nitrogen_ppmw'] = 4.0
COUPLER_options['planet_coresize'] = 0.5

solvevol_vols = ['H2O', 'CO2', 'N2', 'H2', 'CO', 'CH4']
for v in solvevol_vols:
    COUPLER_options[v+"_included"] = 0
    COUPLER_options[v+"_initial_bar"] = 0.0

COUPLER_options["H2O_included"] = 1
COUPLER_options["CO2_included"] = 1
COUPLER_options["N2_included"]  = 1
COUPLER_options["H2_included"]  = 1
COUPLER_options["CO_included"]  = 1
COUPLER_options["CH4_included"] = 1

# COUPLER_options["H2O_initial_bar"] = 1.0
# COUPLER_options["CO2_initial_bar"] = 0.0
# COUPLER_options["N2_initial_bar"]  = 0.0

print("Gravity = %.3f m.s-2" % COUPLER_options["gravity"])



Gravity = 22.304 m.s-2


In [131]:
COUPLER_options["mantle_mass"] = calc_mantle_mass(COUPLER_options)

In [132]:
solvevol_target = solvevol_get_target_from_params(COUPLER_options)
# solvevol_target = solvevol_get_target_from_pressures(COUPLER_options)

print("Targets:")
print(solvevol_target)

Targets:
{'H': 1.24e+21, 'C': 1.24e+21, 'N': 1.5241054973730108e+20}


In [133]:
p_d = solvevol_equilibrium_atmosphere(solvevol_target, COUPLER_options)

print(p_d)

p_bar = {}
for v in volatile_species:
    p_bar[v] = p_d[v+"_atm_bar"]

mubar = 0.0
ptot = 0.0
print("Partial pressures")
for v in solvevol_vols:
    print("    %s : %.3f bar"%(v, p_bar[v]))
    ptot += p_bar[v]
    mubar += p_bar[v]*molar_mass[v]
mubar /= ptot

print("mubar =",mubar)

m_d = {}
for v in solvevol_vols:
    m_d[v] = p_bar[v]*1.0e5 * 4 * 3.14 * (molar_mass[v]/mubar) * COUPLER_options["radius"]**2.0 / COUPLER_options["gravity"]

# print(solvevol_atmosphere_mass(p_bar, COUPLER_options))

print(" ")
print(m_d)

H_kg = (2*m_d["H2O"]/molar_mass["H2O"] + 2*m_d["H2"]/molar_mass["H2"] + 4*m_d["CH4"]/molar_mass["CH4"]) *molar_mass['H']
C_kg = (m_d["CO2"]/molar_mass["CO2"] + m_d["CO"]/molar_mass["CO"] + m_d["CH4"]/molar_mass["CH4"]) * molar_mass['C']
N_kg = m_d["N2"]

print(H_kg, C_kg, N_kg)


{'M_atm': 2.1225087698604377e+21, 'P_surf': 263.99550545393123, 'H2O_atm_bar': 0.3077364717897987, 'H2O_atm_kg': 1.5438725362023365e+18, 'H2O_liquid_kg': 1.1075817990724453e+22, 'H2O_solid_kg': 0.0, 'H2O_total_kg': 1.1077361863260656e+22, 'CO2_atm_bar': 15.51624606722886, 'CO2_atm_kg': 1.901645157301512e+20, 'CO2_liquid_kg': 2.6968332559579703e+20, 'CO2_solid_kg': 0.0, 'CO2_total_kg': 4.598478413259482e+20, 'H2_atm_bar': 0.6906931565382476, 'H2_atm_kg': 3.877406053857657e+17, 'H2_liquid_kg': 0.0, 'H2_solid_kg': 0.0, 'H2_total_kg': 3.877406053857657e+17, 'CH4_atm_bar': 5.432810394121949e-07, 'CH4_atm_kg': 2426723899060.5693, 'CH4_liquid_kg': 286450226241.41724, 'CH4_solid_kg': 0.0, 'CH4_total_kg': 2713174125301.9863, 'CO_atm_bar': 231.4154988774856, 'CO_atm_kg': 1.805082292390178e+21, 'CO_liquid_kg': 7.939651901402958e+20, 'CO_solid_kg': 0.0, 'CO_total_kg': 2.5990474825304737e+21, 'N2_atm_bar': 16.065330337607712, 'N2_atm_kg': 1.2533034617179639e+20, 'N2_liquid_kg': 2.708020356550252e+1

In [134]:
H_mol = H_kg / (1.0e-3)
H_oc  = H_mol / (7.69e22 * 2.0)


print("[H] atm = %.4f" % H_oc)
print("C/H atm = %.4f" % (C_kg/H_kg))

[H] atm = 0.0036
C/H atm = 1473.4915
